In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from topo import SHex
from mesh import uniform_nodes_3d

In [ ]:
L = (1.0,2.0,3.0)
n_elem = 2
vertices, elem_to_vertex, boundary_vertices = \
          uniform_nodes_3d(n_elem,L[0],L[1],L[2])

In [ ]:
topo = SHex()

In [ ]:
[l/n_elem/2.0 for l in L]

In [ ]:
jacb = topo.calc_jacb(vertices[elem_to_vertex])
jacb_inv = topo.calc_jacb_inv(jacb)

jacb[0], jacb_inv[0]

In [ ]:
ref = np.array([[0.,0.,0.], [1,1,1]])
topo.ref_to_phys(vertices[elem_to_vertex], ref)